In [23]:
#Import 
import numpy as np
from skimage import io,color
import matplotlib.pyplot as plt
import math
import pdb
import os
import scipy.signal as sig

def hog(img,prag):

    #PARAMTERS: imagine,threshhold
    #RETURN: GV,GH,histograma gradientilor orientati
    h=img.shape[0]
    w=img.shape[1]
    
    # calculul gradientilor Gh si GV 
    
    Gh = 2*img[0:h-1,0:w-1] - 2*img[1:h,0:w-1] + 4*img[0:h-1,1:w] - 4*img[1:h,1:w]
    Gv = 2*img[0:h-1,0:w-1] - 2*img[0:h-1,1:w] + 4*img[1:h,0:w-1] - 4*img[1:h,1:w]
    
    # calcul magnitudine si orientare folosind Gh si Gv
    Mg = np.sqrt(Gh**2 + Gv**2)
    orientare =(180/math.pi)* np.arctan(Gv/(Gh+0.001))  
    
    #vectorizare magnitudine si gradienti
    Mg = Mg.flatten()
    orientare = orientare.flatten()
    #anularea orientarilor pentru care magnitudinea este mai mica ca pragul ales
    orientare[Mg < prag] = 1800
    orientare = orientare[orientare != 1800]
    
    #calculez histograma de orientari pt 6 bins cu np.histogram
    hist = np.histogram(orientare, bins = 6)[0]
    
    #normez histograma la suma elementelor si adun si un epsilon ca sa
    #evit situatia in care impart la 0 (blocul contine o regiune uniforma)
    hist = hist/(np.sum(hist) + 0.001)
    
    return  Gv,Gh,hist

#verificare Gv,Gh
a= np.array([[100,85,70],[81,80,90],[92,83,50]])
Gv,Gh,hist=hog(a,0.05)
print(Gv,Gh)

path = "D:\MASTER TAID\sem 3\IVOM\lab\Laboratoare\Colocviu\Mari_nr_1"
poze= os.listdir(path)

#masca de ponderi
mask=np.array([[8,3,9],
    [1,4,2],
    [6,5,7]])

#pt. a aplica doar pe patratele rosii din exercitiu
mask_aplicat = np.array([[1,1,1],[0,1,0],[0,1,0]])

dim_bloc = 3
#dim descriptori_lbp = (nr. poze pe care aplicam descriptorul, 6(bins histograma)*9(avem 3x3 griduri))
descriptori_hog=np.zeros([len(poze),6*9])

#parcurge pozele rand pe rand
#citeste,tranf in gray
for k in range(len(poze)):
    img=io.imread("Mari_nr_1/"+ poze[k])
    img = color.rgb2gray(img)

#initializez lista in care calculez descritorii
#calculez pasul pt impartirea in 3x3 ferestre
    lista_histograma_fereastra=[]
    #nr de linii si coloane dupa care se schimba zona 
    val=math.ceil(img.shape[0]/dim_bloc)
    
    h=img.shape[0]
    w=img.shape[1]
    #parcurg imaginea pe blocuri(3x3)
    for i in range(0,h,val):
        for j in range(0,w,val):
            #decupez fiecare ferastra 
            fereastra=img[i:i+val,j:j+val]
            hist_hog = hog(fereastra,0.05)[2]
            #inmultesc histograma hog cu masca
            hist_norm = hist_hog*mask[i//val,j//val]*mask_aplicat[i//val,j//val]
            #pun histograma in lista
            lista_histograma_fereastra.append(hist_norm)
    
    #transform lista de histograme in array
    lista_histograma_fereastra = np.array(lista_histograma_fereastra)
    #pun hostogramele concatenate pe linia corespunzatoare imaginii in
    #descriptori_hog, cel mai simplu este sa folositi flatten() decat 
    #functii de concatenare propriu zise      
    descriptori_hog[k,:]=lista_histograma_fereastra.flatten()

def eucdist(x,y):
    
    #PARAMETERS: cei 2 vectori
    #RETURN: distanta
    distanta = np.sqrt(np.sum(np.square(x - y)))

    return distanta

#initializez matrica de distante    
distante=np.zeros([8,8])

#calculez dist euclidiana intre toti descritorii asociati imaginilor
#inclusiv intre aceeasi descriptori
for k in range(distante.shape[0]):
    for l in range(distante.shape[1]):
        distante[k,l]=eucdist(descriptori_hog[k],descriptori_hog[l])


nr_fete=0

# np.sort - ordoneaza crescator valorile
# np.argsort - ordoneaza indicii in ordinea crescatoare a valorilor efective coresp. fiecarui indice
#prin aceasta metoda, vom obtine in matrice, pe primele 2 coloane, indicii distantelor celor mai mici pt poza respectiva
calc_fete = np.argsort(distante)

#stim ca in folder avem cate 2 poze pt fiecare poza, ordonate: 2xP1, 2xP2, 2xP3, 2xP4
# => pt poza P1 vrem sa obtinem indicii 0,1 (coresp primelor 2 imagini din folder)
#sortam primele 2 coloane pt a obtine mereu indicii in ord 
calc_fete = np.sort(calc_fete[:,:2])
print(calc_fete)
posibile = ['01','23','45','67']

for i in range(calc_fete.shape[0]):
    string = ''
    for j in range(2):
        string+= str(calc_fete[i,j])
    if string in posibile:
        nr_fete += 1
        
        
print('Nr de fete este ', nr_fete)

#############################
########  Intrebare #########
#############################

#Ce este imaginea integrala? De ce avem nevoie de ea?
#Imaginea integrala reprezinta o matrice ce contine suma cumulativa pe linii, apoi pe coloane (utilizand rezultatul de la linii).
#Astfel, primul element al matricei II va fi egal cu primul element al imaginii initiale, iar ultimul va contine suma tuturor valorilor din imaginea initiala.
#Folosim II pentru a calcula suma intensitatilor luminoase (suma pixelilor) din anumite regiuni specifice din imagine, deoarece in coltul din dreapta jos vom avea intotdeauna suma tuturor intensitatilor dintr-o imagine.


[[ 34 -10]
 [ 38 112]] [[ 58 -70]
 [-34 154]]
[[0 1]
 [0 1]
 [2 3]
 [2 3]
 [4 5]
 [4 5]
 [4 6]
 [4 7]]
Nr de fete este  6
